# 1 The objective of this notebook is to analyze the data from the dataset and set up a reference metric with a baseline mode

## Data Atributes/Description:

Attributes:
 * id : unique identifier for candidate (numeric)

 * job_title : job title for candidate (text)

 * location : geographical location for candidate (text)

 * connections: number of connections candidate has, 500+ means over 500 (text)

Output (desired target):
 * fit - how fit the candidate is for the role? (numeric, probability between 0-1)

Keywords: “Aspiring human resources” or “seeking human resources”





In [2]:
#---------Importing libraries---------#

#---Data analysis---#
import pandas as pd
import numpy as np


#---Data visualization---#
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')


#---NLP---#
import re
import string
from collections import Counter

import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#---nltkDownloads---# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from sentence_transformers import SentenceTransformer


#---utils---#
import os
from pathlib import Path


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\joao-lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Packa

In [23]:
#spacy
#lemmatization with spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')

## Step 1: Data Loading and Inspection

Load the dataset into a pandas DataFrame.

Check the dimensions of the dataset (number of rows and columns).

Examine the first few rows of the dataset to get an initial understanding of the data.

Check for any missing values in the dataset.

In [3]:
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw', 'potential-talents - Aspiring human resources - seeking human resources.csv')

df = pd.read_csv(raw_data_path)

df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
df.dtypes

id              int64
job_title      object
location       object
connection     object
fit           float64
dtype: object

In [5]:
#onyl object cols
df[df.columns[df.dtypes == 'object']].describe()

,job_title,location,connection
count,104,104,104
unique,52,41,33
top,2019 C.T. Bauer College of Business Graduate (...,Kanada,500+
freq,7,12,44


In [6]:
df.describe(include=object)

,job_title,location,connection
count,104,104,104
unique,52,41,33
top,2019 C.T. Bauer College of Business Graduate (...,Kanada,500+
freq,7,12,44


In [7]:
#checking for missing values
df.isnull().sum()

id              0
job_title       0
location        0
connection      0
fit           104
dtype: int64

## Step 2: Data Cleaning and Preprocessing

Handle missing values, if any, by either imputing them or removing rows/columns with missing values, depending on the situation.

Convert categorical variables into appropriate data types (e.g., convert 'month' and 'contact' to categorical data type).

Encode binary categorical variables ('default', 'housing', 'loan', and 'y') into numeric format (0 and 1) for modeling purposes.

In [8]:
# drop the fit row
df.drop(columns=['fit'], inplace=True)

df.head()

,id,job_title,location,connection
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+


In [11]:
#setting the index to be the id
df.set_index('id', inplace=True)

In [12]:
#checking for duplicated rows
df.duplicated().sum()

51

In [14]:
#drop duplicated rows
df.drop_duplicates(inplace=True)
df.shape

(53, 3)

In [15]:
#checking for na or null values
df.isnull().sum()

job_title     0
location      0
connection    0
dtype: int64

In [17]:
df_clean = df.copy()


# Text Preprocessing

Col - job tittles

In [34]:
#text preprocessing function
def text_cleaner(text):
    #lowercase
    text = text.lower()
    #remove special characters and digits
    text = re.sub(r"(\d|\W)+"," ",text)
    #remove single characters from the start, from the middle and from the end of the text
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]$', ' ', text)
    #removing numbers
    text = re.sub(r'\d+', '', text)

    #remove stopwords
    stop_words = nltk.corpus.stopwords.words('english')
    text_tokens = nltk.word_tokenize(text)
    text = [word for word in text_tokens if word not in stop_words]
    
    #lemmatization with spacy
    text = nlp(' '.join(text))
    text = [word.lemma_ for word in text]
    text = ' '.join(text)
    #remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    



    return text

In [35]:
df_clean['job_title'] = df_clean['job_title'].apply(lambda x: text_cleaner(x))
df_clean.head(15)

,job_title,location,connection
id,,,
1,bauer college business graduate magna cum laud...,"Houston, Texas",85
2,native english teacher epik english program korea,Kanada,500+
3,aspire human resource professional,"Raleigh-Durham, North Carolina Area",44
4,people development coordinator ryan,"Denton, Texas",500+
5,advisory board member celal bayar university,"İzmir, Türkiye",500+
6,aspire human resource specialist,Greater New York City Area,1
7,student humber college aspire human resource g...,Kanada,61
8,hr senior specialist,San Francisco Bay Area,500+
10,seek human resource hris generalist position,Greater Philadelphia Area,500+


In [40]:
# visualizing the differences that cleaning made
print(df_clean['job_title'].iloc[0])
print(df['job_title'].iloc[0])

bauer college business graduate magna cum laude aspire human resource professional
2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional


Basic NLTK


In [17]:
example = df['job_title'][20]
print(example)
print(type(example))

Aspiring Human Resources Professional
<class 'str'>


In [27]:
tokens = nltk.tokenize.word_tokenize(example, language='english', preserve_line=False)
tokens

['Aspiring', 'Human', 'Resources', 'Professional']

## experimenting with the vader model on raw data to get the sentiment analysis of the job tittle


In [30]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [32]:
sia.polarity_scores(example)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [37]:
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.loc[i, 'sentiment'] = sia.polarity_scores(row['job_title'])['compound']
    df.loc[i, 'sentiment_negative'] = sia.polarity_scores(row['job_title'])['neg']
    df.loc[i, 'sentiment_neutral'] = sia.polarity_scores(row['job_title'])['neu']
    df.loc[i, 'sentiment_positive'] = sia.polarity_scores(row['job_title'])['pos']

  0%|          | 0/104 [00:00<?, ?it/s]

In [39]:
df.sort_values(by='sentiment', ascending=False).head(10)

,id,job_title,location,connection,fit,sentiment,sentiment_negative,sentiment_neutral,sentiment_positive
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,NaN,0.8442,0.0,0.521,0.479
91,92,Seeking employment opportunities within Custom...,"Torrance, California",64,NaN,0.7003,0.0,0.547,0.453
85,86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,NaN,0.6369,0.0,0.704,0.296
103,104,Director Of Administration at Excellence Logging,"Katy, Texas",500+,NaN,0.6249,0.0,0.549,0.451
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN,0.5719,0.0,0.575,0.425
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,0.4767,0.0,0.617,0.383
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,NaN,0.4404,0.0,0.734,0.266
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,0.3818,0.0,0.536,0.464
93,94,Seeking Human Resources Opportunities. Open t...,Amerika Birleşik Devletleri,415,NaN,0.3818,0.0,0.755,0.245
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,0.3818,0.0,0.536,0.464


# 

## Roberta Pretrained Model

* transformer model accounts for the words but also the context of the words in the sentence